In [29]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime


In [30]:
backups=False

if backups == True:
    # add backup of BICC

    source_folder = '/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025'
    backup_root = '/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/Backups'

    # Create timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_folder = os.path.join(backup_root, f'BICC_Backup_{timestamp}')

    # Make sure the backup root exists
    os.makedirs(backup_root, exist_ok=True)

    # Copy the folder
    shutil.copytree(source_folder, backup_folder)

    print(f"Backup completed: {backup_folder}")
else:
    print("Backups not enabled.")


Backups not enabled.


In [31]:
# load svensson links
sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/Svensson Links NGRIP-NEEM-GIP-GISP-EDML-EDC-WD-DJ-TALDICE.xls',
                         sheet_name=1, nrows=305, usecols=[0,2,3,4,5,6,7,], names=['NGRIP', 'GRIP', 'GISP2', 'EDML', 'EDC', 'WDC', 'DF'])

#load cores list from params
params = '/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025/parameters.yml'
with open(params, 'r') as f:
    first_line = f.readline()
params_load = yaml.safe_load(first_line)
list_sites = params_load['list_sites']

# get all link combos
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

#get all combos possible from the svensson links
subset = list(sv_links.columns)
valid_pairs = [p for p in pairs if all(site in subset for site in p.split('-'))]

In [32]:
#now, create all the link files
sv_synchros = {}
sv_comments = {}
for pair in valid_pairs:
    site_a, site_b = pair.split('-')

    # Drop NaN rows to get only valid shared tiepoints
    sv_synchros[pair] = sv_links[[site_a, site_b]].dropna()
    sv_synchros[pair].columns = ['depth1', 'depth2']
    sv_synchros[pair]['age_unc'] = 5
    sv_synchros[pair]['comment'] = np.nan  # Initialize age column with NaN

    sv_comments[pair] = ['#Includes tiepoints from Svensson Links Unpub 2025']

#now load existing synchros
os.chdir('/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025')
existing_synchros = {}
existing_comments = {}

for pair in valid_pairs:
    folder_path = pair
    file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")

    os.makedirs(folder_path, exist_ok=True)

    if os.path.exists(file_path):

        comments = []
        with open(file_path, 'r') as f: # store existing comments to be re-added
            for line in f:
                if line.startswith('#'):
                    comments.append(line.rstrip('\n'))  # keeps tabs intact  # store comment lines without newline
        existing_comments[pair] = comments

        synchros = pd.read_csv(file_path, sep='\t', comment='#')
        existing_synchros[pair] = synchros
        print(f"Loaded {pair}: {len(synchros)} rows")
    else:
        print(f"Skipped {pair} — file not found")

Loaded DF-EDC: 1401 rows
Skipped DF-EDML — file not found
Loaded DF-NGRIP: 215 rows
Skipped DF-GRIP — file not found
Skipped DF-WDC — file not found
Skipped DF-GISP2 — file not found
Loaded EDC-EDML: 175 rows
Skipped EDC-NGRIP — file not found
Skipped EDC-GRIP — file not found
Loaded EDC-WDC: 566 rows
Skipped EDC-GISP2 — file not found
Loaded EDML-NGRIP: 372 rows
Skipped EDML-GRIP — file not found
Loaded EDML-WDC: 942 rows
Skipped EDML-GISP2 — file not found
Loaded NGRIP-GRIP: 1030 rows
Loaded NGRIP-WDC: 192 rows
Loaded NGRIP-GISP2: 713 rows
Loaded GRIP-WDC: 98 rows
Loaded GRIP-GISP2: 801 rows
Loaded WDC-GISP2: 170 rows


In [33]:
tolerance = 0.10  # meters

for pair in valid_pairs:

    if pair in existing_synchros:
        
        overlaps = 0
        new_ties = 0

        existing = existing_synchros[pair].copy(deep=True)
        new = sv_synchros[pair].copy(deep=True)

        # Start with Svensson links (priority)
        combined = new.copy(deep=True)

        # Append only existing tiepoints that are NOT close to any new tiepoint
        for idx, row in existing.iterrows():
            # Check if any row in new is within tolerance on EITHER column
            mask = (
                (abs(new.iloc[:, 0] - row.iloc[0]) <= tolerance) |
                (abs(new.iloc[:, 1] - row.iloc[1]) <= tolerance)
            )
            if not mask.any():
                combined = pd.concat([combined, row.to_frame().T], ignore_index=True)
                new_ties+=1
            else:
                overlaps += 1



        folder_path = pair
        file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")
        os.makedirs(folder_path, exist_ok=True)

        # --- Gather comments ---
        new_comments = sv_comments.get(pair, [])

        with open(file_path, 'w') as f:
            # Write new comments first
            for line in new_comments:
                f.write(f"{line}\n")
            # Add original comments
            for line in existing_comments.get(pair, []):
                f.write(f"{line}\n")
            # Write merged DataFrame below
            combined.to_csv(f, sep='\t', index=False)
        print(f"Saved merged synchro for {pair} with rows to {file_path}\nThere were {overlaps} overlaps, and {new_ties} new ties.")

    else:
        folder_path = pair
        file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")
        os.makedirs(folder_path, exist_ok=True)

        combined = sv_synchros[pair].copy(deep=True) # svensson links load
        new_comments = sv_comments.get(pair, [])

        with open(file_path, 'w') as f:
            # Write new comments first
            for line in new_comments:
                f.write(f"{line}\n")
            # Write merged DataFrame below
            combined.to_csv(f, sep='\t', index=False)

        print(f"No existing synchros for {pair}, saving new synchro to {file_path}")


Saved merged synchro for DF-EDC with rows to DF-EDC/iceice_synchro_horizons.txt
There were 165 overlaps, and 1236 new ties.
No existing synchros for DF-EDML, saving new synchro to DF-EDML/iceice_synchro_horizons.txt
Saved merged synchro for DF-NGRIP with rows to DF-NGRIP/iceice_synchro_horizons.txt
There were 215 overlaps, and 0 new ties.
No existing synchros for DF-GRIP, saving new synchro to DF-GRIP/iceice_synchro_horizons.txt
No existing synchros for DF-WDC, saving new synchro to DF-WDC/iceice_synchro_horizons.txt
No existing synchros for DF-GISP2, saving new synchro to DF-GISP2/iceice_synchro_horizons.txt
Saved merged synchro for EDC-EDML with rows to EDC-EDML/iceice_synchro_horizons.txt
There were 94 overlaps, and 81 new ties.
No existing synchros for EDC-NGRIP, saving new synchro to EDC-NGRIP/iceice_synchro_horizons.txt
No existing synchros for EDC-GRIP, saving new synchro to EDC-GRIP/iceice_synchro_horizons.txt
Saved merged synchro for EDC-WDC with rows to EDC-WDC/iceice_synchro